## Import necessary modules

In [2]:
!pip install --upgrade pip
!pip install gensim
!pip install pandas==2.2.2
!pip install scikit-learn==1.5.0
!pip install tqdm
!pip install scipy==1.11.1

  Using cached scikit_learn-1.5.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached scikit_learn-1.5.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.3 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.4.2
    Uninstalling scikit-learn-1.4.2:
      Successfully uninstalled scikit-learn-1.4.2
  Using cached scipy-1.11.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (59 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.2/36.2 MB 1.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.0
    Uninstalling scipy-1.13.0:
      Successfully uninstalled scipy-1.13.0


In [3]:
import numpy as np

import gensim.downloader as api
from gensim.models import KeyedVectors

from utils import DatasetReaderit 

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC

from sklearn.metrics import classification_report

from tqdm.notebook import tqdm

ImportError: cannot import name 'triu' from 'scipy.linalg' (/home/pc/lib/python3.11/site-packages/scipy/linalg/__init__.py)

## Load embeddings

In [2]:
# embeddings = api.load('word2vec-google-news-300')
# embeddings.save('weights/word2vec.kv')  

embeddings: KeyedVectors = KeyedVectors.load('weights/word2vec.kv')

## Load dataset

In [3]:
reader = DatasetReader()
data = reader.read_from_file('../data/csv/clean_data.csv')
data

,Questions,Topic
0,define the term brand,marketing_mix_and_strategy
1,explain one risk jack ma may have taken when s...,entreprenuers_and_leaders
2,analyse two factors that may have increased de...,market
3,discuss if profit maximisation is the main bus...,entreprenuers_and_leaders
4,assess the advantages of a paternalistic style...,managing_people
...,...,...
304,evaluate the likely value of each of the follo...,meeting_customer_needs
307,evaluate the likely value to mike watson of us...,meeting_customer_needs
310,briefly explain two reasons why levi roots con...,meeting_customer_needs
313,assess the likely implications for reggae regg...,meeting_customer_needs


## Data preprocessing

In [4]:
vectors = data.to_numpy()
vectors

array([['define the term brand', 'marketing_mix_and_strategy'],
       ['explain one risk jack ma may have taken when setting up the alibaba group',
        'entreprenuers_and_leaders'],
       ['analyse two factors that may have increased demand for the alibaba group',
        'market'],
       ['discuss if profit maximisation is the main business objective of the alibaba group',
        'entreprenuers_and_leaders'],
       ['assess the advantages of a paternalistic style of leadership for a business such as the alibaba group',
        'managing_people'],
       ['define the term product innovation ', 'meeting_customer_needs'],
       ['if nintendo reduces its console prices by 8% calculate the estimated percentage change in demand. you are advised to show your working',
        'market'],
       ['analyse two different pricing strategies that nintendo might use to compete in the games console industry',
        'marketing_mix_and_strategy'],
       ['discuss how the boston matrix may

In [5]:
encoder = LabelEncoder().fit(vectors[:, 1])

encoded = np.stack([vectors[:, 0], encoder.transform(vectors[:, 1])], axis=1)
encoded

array([['define the term brand', 3],
       ['explain one risk jack ma may have taken when setting up the alibaba group',
        0],
       ['analyse two factors that may have increased demand for the alibaba group',
        2],
       ['discuss if profit maximisation is the main business objective of the alibaba group',
        0],
       ['assess the advantages of a paternalistic style of leadership for a business such as the alibaba group',
        1],
       ['define the term product innovation ', 4],
       ['if nintendo reduces its console prices by 8% calculate the estimated percentage change in demand. you are advised to show your working',
        2],
       ['analyse two different pricing strategies that nintendo might use to compete in the games console industry',
        3],
       ['discuss how the boston matrix may help companies such as nintendo develop their product portfolio',
        3],
       ['assess the benefits of a matrix organisational structure for a business

In [6]:
def process_sentence(sentence, embedder):
    words = sentence.split(' ')
    res = []
    for w in words:
        if w in embedder:
            res.append(embedder.get_vector(w))
    return np.array(res).mean(axis=0)

In [11]:
X = []
y = []

for sample in encoded:
    X.append(process_sentence(sample[0], embeddings))
    y.append(sample[1])

X, y = np.array(X), np.array(y)
X.shape, y.shape

((264, 300), (264,))

## Splitting the data

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((211, 300), (53, 300), (211,), (53,))

## Training the model

In [17]:
svc = SVC()
svc.fit(X_train, y_train)

SVC()

## Evaluating the model

In [18]:
print(classification_report(y_test, svc.predict(X_test)))

              precision    recall  f1-score   support

           0       1.00      0.50      0.67         8
           1       0.86      0.86      0.86         7
           2       1.00      0.75      0.86         8
           3       0.67      0.86      0.75        14
           4       0.61      0.69      0.65        16

    accuracy                           0.74        53
   macro avg       0.83      0.73      0.76        53
weighted avg       0.78      0.74      0.74        53

